<a href="https://colab.research.google.com/github/talhadar1/real_estate_price_forecasting/blob/master/real_estate_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
# drive.mount('/content/drive')
drive.mount("/content/drive", force_remount=True)


Mounted at /content/drive


### Enabling and testing the TPU

First, you'll need to enable TPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select TPU from the Hardware Accelerator drop-down



# **Import libraries and files:**

In [0]:
import os 
# Import libraries necessary for this project
import numpy as np
import pandas as pd
import pandas_profiling
from pandas_profiling import ProfileReport
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import learning_curve
folder_path = '/content/drive/My Drive/seminar_data/'
file_name = 'TransactionsXLSsince2008.xlsx'


# **import data:**

In [2]:
#import dataset from xlsx file
orig_df = pd.read_excel('/content/drive/My Drive/TransactionsXLSsince2008.xlsx')
# Success
print("Israel housing dataset has {} data points with {} variables each.".format(*orig_df.shape))

Israel housing dataset has 664606 data points with 18 variables each.


# **Data Exploration:**
In the first section of the project, we will make an exploratory analysis of the dataset and provide some observations.

Initially we could make the following intuitive assumptions for each feature:


*   Property area with a bigger square meters (higher ‘shetachneto’ value) will worth more. Usually Property with more shetachneto  can fit more people, so it is reasonable that they cost more money. They are directly proportional variables.
*   Neighborhoods with lower Socioeconomic Index (lower ‘soccesso’ value) will worth less. If the Socioeconomic Index value is low at a neighborhood , it is likely that they have low purchasing power and therefore, they houses will cost less. They are directly proportional variables.
*  Property built later (higher ‘Year of Construction’ value) will worth more. Usually later built property will have newer construction and will feature more innovative features (elevator, shelter, green building, underground parking, etc.). They are directly proportional variables.

In [3]:
ProfileReport(orig_df)

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,18
Number of observations,664606
Total Missing (%),5.7%
Total size in memory,91.3 MiB
Average record size in memory,144.0 B
Numeric,12
Categorical,6
Boolean,0
Date,0
Text (Unique),0
Rejected,0




*   tat_helka has a high cardinality: 13916 distinct values Warning

*   date_transaction has a high cardinality: 3825 distinct values Warning

*   city_hebrew has a high cardinality: 784 distinct values Warning

*   rechov has a high cardinality: 13973 distinct values Warning 

---

1.   Note that the transaction date appears as both date_transaction and split date year, month and day. The full date does not give us good data for days and months so we will lower it. And leave the three properties of the second option.
2.  'rechov' - This variable represents the street name, the street name has no empirical significance to the price of the apartment directly and we see that there is no correlation between the street name and the price of the property.  We will drop this property!
3.  'city_hebrew' - This variable represents the name of the city, the name of the city has no empirical significance to the price of the apartment directly and we see that there is no correlation at all between the name of the city and the price of the property. There may be identical apartments with their characteristics in the same city but with different prices. We will drop this property!
4. 'tat_helka' - This variable represents the number of subdivisions where the property is located within the block (usually 2-4 properties),
A sub-plot has no direct empirical significance to the price, but only when it comes in combination with the block and plot where it exists.
We will drop this property!

5. 'bayit' & 'dira' - This variable represents the building number and apartment number on the street where the property is located respectively,
Similarly to the street name, this variable is also not directly correlated with the price, We will drop this properties!


In [0]:
#Drop any unnecessary features
df = orig_df.drop(['rechov', 'date_transaction', 'city_hebrew', 'tat_helka', 'bayit', 'dira'], axis=1).dropna(how='all')

Validate the dimensions of the dataframe:

In [5]:
# dataframe.size - number of cells
size = orig_df.size 
  
# dataframe.shape - shape[0]=number of rows  | shape[1]=number of columns
shape = orig_df.shape 

# printing size and shape 
print("number of cells = {}\nShape ={}\#ROWS x #COLUMNS = {}". format(size, shape, shape[0]*shape[1])) 

number of cells = 11962908
Shape =(664606, 18)\#ROWS x #COLUMNS = 11962908


#Handling missing & duplicate data:

*   floors has 85099 / 12.8% missing values Missing

*   dirotbnyn has 179834 / 27.1% missing values Missing

*   soceco has 87191 / 13.1% missing values Missing

*   Dataset has 332 duplicate rows Warning

In [0]:
# Drop the 5-th percentile
df = df[df['year']>1950]

# Change all 'floors' with NaN to - 5 (Median of the floors column)
df['floors'] = df['floors'].fillna(5)

# Change all 'dirotbnyn' with NaN to - 5 (Median of the dirotbnyn column)
df['dirotbnyn'] = df['dirotbnyn'].fillna(18)

# Drop all rows with NaN at 'soceco'
df['soceco'] = df['soceco'].fillna(0.16788)
# df['soceco'] = df['soceco'].dropna(how='all')

# drop all duplicate rows in the dataframe
df.drop_duplicates(keep=False,inplace=True) 


This cell is exuted only because of the lack of hardware resources on the machine we run the project on.

In [0]:
#take only precent of the data set for training and testing. 
data_set_precent = 0.5
df = df.sample(frac=data_set_precent)

In the variable "gush_helka" there are 13916 different values ​​by dropping the "helka" we can reduce by 50%. Because this variable was a categorical variable we would need every value in which the dummy variable and therefore this reduction would improve the runtime performance of the model.

if the value was "002023-432" it will be "002023" after this cell execution.

In [8]:
df['gush'] = df['gush_helka'].apply(lambda x: x.split('-')[0])
df = df.drop('gush_helka', axis=1)
df.head()

,price_sheqel,year_of_construction,floor,shetachneto,type,floors,dirotbnyn,year,month,day,soceco,gush
232492,2808721.0,2015.0,5.0,76.0,דירה בבית קומות,6.0,18.0,2015.0,7.0,26.0,0.167880,006948
346537,175000.0,1960.0,2.0,23.0,דירה בבית קומות,3.0,18.0,2013.0,5.0,8.0,-0.672111,010864
423352,520000.0,1990.0,0.0,80.0,קוטג' דו-משפחתי,5.0,18.0,2012.0,4.0,12.0,0.167880,073654
163396,800000.0,1981.0,4.0,79.0,דירה בבית קומות,7.0,28.0,2016.0,10.0,9.0,-0.570653,018018
245327,1172200.0,1970.0,1.0,57.0,דירה בבית קומות,2.0,18.0,2015.0,5.0,20.0,1.723426,011191


In [9]:
ProfileReport(df)

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,13
Number of observations,327100
Total Missing (%),0.0%
Total size in memory,32.4 MiB
Average record size in memory,104.0 B
Numeric,11
Categorical,2
Boolean,0
Date,0
Text (Unique),0
Rejected,0


**Dataset It has no missing values ​​and no duplicates rows**


'gush' and 'type' both are **categorical variables**.

There is no empirical meaning to price proportionally. If one neighborhood number is '7000' and a second neighborhood number is '9000' this does not mean that in the second neighborhood property prices are higher than in the first neighborhood and vice versa.

Similarly, if the type of property we were an apartment building does not mean that its price will be higher or lower than the price of a private house.

**Therefore we will make these variables a dummy variable.**

In [10]:
df = pd.get_dummies(df.dropna())

# dataframe.size - number of cells
size = df.size 
# dataframe.shape - shape[0]=number of rows  | shape[1]=number of columns
shape = df.shape 
# printing size and shape 
print("number of cells = {}\nShape ={}\#ROWS x #VARIABLES = {}". format(size, shape, shape[0]*shape[1]))

# Creating the same dataframe without the socioeconomic property of the property.
df_2 = df.drop('soceco', axis=1)

number of cells = 1959227170
Shape =(327083, 5990)\#ROWS x #VARIABLES = 1959227170


dataset has now 5957 different variables(columns).
10 of them are original and the other are dummy variables
  

#Developing a Model#
Defining a Performace Metric:
we will calculate the coefficient of determination, R², to quantify the model’s performance. The coefficient of determination for a model is a useful statistic in regression analysis, as it often describes how “good” that model is at making predictions.

In [0]:
# Import 'r2_score'

from sklearn.metrics import r2_score

def performance_metric(y_true, y_predict):
    """ Calculates and returns the performance score between 
        true (y_true) and predicted (y_predict) values based on the metric chosen. """
    
    score = r2_score(y_true, y_predict)
    
    # Return the score
    return score

##Shuffle and Split Data:




In [12]:
#Tests data is acording to the 'Pareto principle' is  80% of the data is for training and the rest 20% for testing
#https://en.wikipedia.org/wiki/Pareto_principle
test_frac = .2 
#(1) Generating dataframes for training and testing:
train_df, test_df, y_train, y_test = train_test_split(df.drop('price_sheqel', axis=1), df['price_sheqel'], test_size=test_frac,random_state = 42)

#(2) Generating dataframes for training and testing without the 'soceco' feature:
train_df2, test_df2, y_train2, y_test2 = train_test_split(df_2.drop('price_sheqel', axis=1), df_2['price_sheqel'], test_size=test_frac,random_state = 42)

#We don't need to save the original dataframes because we already split them. Freeing them from memory will contribute to program performance.
# del df , df_2
print("Training and testing split was successful.")

Training and testing split was successful.


##Training and Testing#

### **Random Forest Regressor :**

In [0]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
trees_num = 100

#Each leaf will contain at least 'min_samples_in_a_leaf' sampels. it will help us control the depth of the trees.
min_samples_in_a_leaf = 10

# Instantiate 2 models with 'trees_num' decision trees each
rf = RandomForestRegressor(n_estimators = trees_num, random_state = 42, verbose=100, min_samples_leaf=min_samples_in_a_leaf, n_jobs=-1)
rf_no_soceco = RandomForestRegressor(n_estimators = trees_num, random_state = 42, verbose=100, min_samples_leaf=min_samples_in_a_leaf, n_jobs=-1)

####Train the model on training data


In [14]:
#Train the Random Forest models on training data:
rf.fit(train_df, y_train)
rf_no_soceco.fit(train_df2, y_train2)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 40 concurrent workers.
building tree 1 of 100building tree 2 of 100building tree 3 of 100building tree 4 of 100building tree 5 of 100building tree 6 of 100building tree 7 of 100building tree 8 of 100building tree 9 of 100building tree 10 of 100building tree 11 of 100building tree 12 of 100building tree 13 of 100building tree 14 of 100building tree 15 of 100building tree 16 of 100building tree 17 of 100building tree 18 of 100
building tree 19 of 100building tree 20 of 100building tree 21 of 100building tree 22 of 100building tree 23 of 100building tree 24 of 100building tree 25 of 100building tree 26 of 100building tree 27 of 100building tree 28 of 100building tree 29 of 100building tree 30 of 100building tree 31 of 100building tree 32 of 100building tree 33 of 100building tree 34 of 100building tree 35 of 100building tree 36 of 100building tree 37 of 100building tree 38 of 100building tree 39 of 100building tree 40 of 100







RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=10,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=-1, oob_score=False,
                      random_state=42, verbose=100, warm_start=False)



####Test the model on the the test data

In [15]:
# Use the forest's predict method on the test data
from sklearn import metrics
predictions  = rf.predict(test_df)
print('Predictions statistics random forest reggresion:')

# Calculate the absolute errors
errors = abs(predictions - y_test)

# Print out the mean absolute error (mae)
print('Mean Absolute Error (MAE):', metrics.mean_absolute_error(y_test, predictions))
print('Mean Squared Error (MSE):', metrics.mean_squared_error(y_test, predictions))
print('Root Mean Squared Error (RMSE):', np.sqrt(metrics.mean_squared_error(y_test, predictions)))
mape = np.mean(np.abs((y_test - predictions) / y_test))
print('Mean Absolute Percentage Error (MAPE):', round(mape * 100, 2))
print('Accuracy:', round(100*(1 - mape), 2))

[Parallel(n_jobs=40)]: Using backend ThreadingBackend with 40 concurrent workers.
[Parallel(n_jobs=40)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=40)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=40)]: Done   3 tasks      | elapsed:    0.0s
[Parallel(n_jobs=40)]: Done   4 tasks      | elapsed:    0.0s
[Parallel(n_jobs=40)]: Done   5 tasks      | elapsed:    0.0s
[Parallel(n_jobs=40)]: Done   6 tasks      | elapsed:    0.0s
[Parallel(n_jobs=40)]: Done   7 tasks      | elapsed:    0.1s
[Parallel(n_jobs=40)]: Done   8 tasks      | elapsed:    0.1s
[Parallel(n_jobs=40)]: Done   9 tasks      | elapsed:    0.1s
[Parallel(n_jobs=40)]: Done  10 tasks      | elapsed:    0.1s
[Parallel(n_jobs=40)]: Done  11 tasks      | elapsed:    0.1s
[Parallel(n_jobs=40)]: Done  12 tasks      | elapsed:    0.1s
[Parallel(n_jobs=40)]: Done  13 tasks      | elapsed:    0.1s
[Parallel(n_jobs=40)]: Done  14 tasks      | elapsed:    0.1s
[Parallel(n_jobs=40)]: Done  15 tasks      | elaps

In [16]:
# Use the forest's predict method on the test data
from sklearn import metrics
predictions_no_soceco = rf_no_soceco.predict(test_df2)
print('Predictions_no_soceco statistics random forest reggresion without "soceco":')
# Calculate the absolute errors
errors = abs(predictions_no_soceco - y_test2)
# Print out the mean absolute error (mae)
print('Mean Absolute Error (MAE):', metrics.mean_absolute_error(y_test, predictions))
print('Mean Squared Error (MSE):', metrics.mean_squared_error(y_test, predictions))
print('Root Mean Squared Error (RMSE):', np.sqrt(metrics.mean_squared_error(y_test, predictions)))
mape = np.mean(np.abs((y_test - predictions) / y_test))
print('Mean Absolute Percentage Error (MAPE):', round(mape * 100, 2))
print('Accuracy:', round(100*(1 - mape), 2))


[Parallel(n_jobs=40)]: Using backend ThreadingBackend with 40 concurrent workers.
[Parallel(n_jobs=40)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=40)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=40)]: Done   3 tasks      | elapsed:    0.0s
[Parallel(n_jobs=40)]: Done   4 tasks      | elapsed:    0.0s
[Parallel(n_jobs=40)]: Done   5 tasks      | elapsed:    0.1s
[Parallel(n_jobs=40)]: Done   6 tasks      | elapsed:    0.1s
[Parallel(n_jobs=40)]: Done   7 tasks      | elapsed:    0.1s
[Parallel(n_jobs=40)]: Done   8 tasks      | elapsed:    0.1s
[Parallel(n_jobs=40)]: Done   9 tasks      | elapsed:    0.1s
[Parallel(n_jobs=40)]: Done  10 tasks      | elapsed:    0.1s
[Parallel(n_jobs=40)]: Done  11 tasks      | elapsed:    0.1s
[Parallel(n_jobs=40)]: Done  12 tasks      | elapsed:    0.1s
[Parallel(n_jobs=40)]: Done  13 tasks      | elapsed:    0.1s
[Parallel(n_jobs=40)]: Done  14 tasks      | elapsed:    0.1s
[Parallel(n_jobs=40)]: Done  15 tasks      | elaps

####Feature importance

In [17]:
importances = list(zip(train_df.columns.to_list(), rf.feature_importances_))
epsilon = 0.0001
count = 0
for col, val in sorted(importances, key=lambda x: x[1], reverse=True):
    if val > epsilon:
      count = count + 1
      print(f'{col}: {val}')
print(count)

shetachneto: 0.368715336393408
soceco: 0.2655825738641845
year_of_construction: 0.07772949054093595
year: 0.0662925439619099
floors: 0.0504937581092814
floor: 0.03950589312307701
dirotbnyn: 0.026526906276863646
day: 0.014155467394533685
month: 0.011354906753438896
gush_007104: 0.008898016659466837
gush_006901: 0.004780496006451528
type_דירה בבית קומות: 0.003923106484555634
gush_006909: 0.0038924842538717026
gush_007661: 0.002353425712145075
gush_006217: 0.0022440404623422705
gush_007101: 0.0018738960294156076
gush_006914: 0.0018635456926240901
gush_010423: 0.001547678006153274
gush_006668: 0.0013479507329110336
gush_006422: 0.0012489402774205208
type_קוטג' דו-משפחתי: 0.0011296298865457884
gush_006108: 0.0010631631343932027
gush_006111: 0.0010016239940903464
type_דירת גן: 0.0009968070388405418
gush_005611: 0.0009900163229979956
gush_007621: 0.0009314583464791113
gush_006916: 0.0008904921478566771
type_בית בודד: 0.0008378026713942936
gush_006213: 0.0008329297159158379
gush_030146: 0.0008

####Save model to file:

In [21]:
import joblib
joblib_file = '/content/drive/My Drive/RandomForestRegressor_model.pkl'
joblib_file_no_soceco = '/content/drive/My Drive/RandomForestRegressor_model_no_soceco.pkl'
joblib.dump(rf,joblib_file)
joblib.dump(rf_no_soceco,joblib_file_no_soceco)

['/content/drive/My Drive/RandomForestRegressor_model_no_soceco.pkl']

In [0]:
# Use the line below  to get all transactions at the same block ('gush')
# original.dropna()[original['gush_helka'].dropna().str.contains('009126')].sort_values('price_sheqel')



### **Linear Regression**

####Train the model on training data

In [23]:
from sklearn.linear_model import LinearRegression
# Create model
reg = LinearRegression()    
# Train model
reg.fit(train_df, y_train) 

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

####Test the model on the test data

In [24]:
# Predict method on the test data
predictions = reg.predict(test_df)

# Use the forest's predict method on the test data
from sklearn import metrics
predictions  = reg.predict(test_df)
print('Predictions statistics linear-reggresion model:')

# Calculate the absolute errors
errors = abs(predictions - y_test)/y_test

# Print out the mean absolute error (mae)
print('Mean Absolute Error (MAE):', metrics.mean_absolute_error(y_test, predictions))
print('Mean Squared Error (MSE):', metrics.mean_squared_error(y_test, predictions))
print('Root Mean Squared Error (RMSE):', np.sqrt(metrics.mean_squared_error(y_test, predictions)))
mape = np.mean(np.abs((y_test - predictions) / y_test))
print('Mean Absolute Percentage Error (MAPE):', round(mape * 100, 2))
print('Accuracy:', round(100*(1 - mape), 2))

Predictions statistics linear-reggresion model:
Mean Absolute Error (MAE): 39899383183.23649
Mean Squared Error (MSE): 2.2336226055420305e+25
Root Mean Squared Error (RMSE): 4726121671669.098
Mean Absolute Percentage Error (MAPE): 749473424.16
Accuracy: -749473324.16


####Save model to file:

In [26]:
import joblib
joblib_file =  '/content/drive/My Drive/LinearRegression.pkl'
joblib.dump(reg,joblib_file)

['/content/drive/My Drive/LinearRegression.pkl']

## **SGD Regression**

In [0]:
from sklearn.linear_model import SGDRegressor
sgd_reg = SGDRegressor()
sgd_reg.fit(train_df, y_train)

In [0]:
#Predict method on the test data
predictions = sgd_reg.predict(test_df)

In [0]:
# Calculate the absolute errors
errors = abs(predictions - y_test)
# Print out the mean absolute error (mae)
print(f'Mean Absolute Error: {round(np.mean(errors), 2)} shekel')

In [0]:
test_df.head(2)
sgd_reg.predict(test_df.head(2))